## Reading phototransistor with a labJack T7\
Uses 1 analog input (AINs) to read the data.

Craig Lage - Jan 24, 2023

In [ ]:
import sys
import time, datetime
import numpy as np
from astropy.time import Time, TimeDelta

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
ljm.openS?

In [ ]:
# Open LabJack T7
handle = ljm.openS("T7", "USB", "auxtel_dome_flat") 

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

In [ ]:
# Open LabJack T4
handle = ljm.openS("T4", "ETHERNET", "139.229.170.151")  

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Setup and call eWriteNames to configure AIN0 and AIN1 on the LabJack.
# LabJack T7 and other devices configuration

# AIN0 and AIN1:
#   Negative channel = single ended (199)
#   Range: +/-10.0 V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0)
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US",
        "AIN2_NEGATIVE_CH", "AIN2_RANGE", "AIN2_RESOLUTION_INDEX", "AIN2_SETTLING_US"]
aValues = [199, 10.0, 0, 0,
           199, 10.0, 0, 0,
            199, 10.0, 0, 0]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

In [ ]:
print("\nSet configuration:")
for i in range(numFrames):
    print("    %s : %f" % (names[i], aValues[i]))


In [ ]:
# Read AIN0(Adam Ain3 - High pressure side) and AIN1 (Adam Ain4 - Low pressure side)

loopAmount = 10
# Read AIN's from the LabJack with eReadNames in a loop.
names = ["AIN0", "AIN1", "AIN2"]
numFrames = 3
intervalHandle = 1
ljm.startInterval(intervalHandle, 50000)  # Delay between readings (in microseconds)
plotResults = []
start = time.time()
for n in range(loopAmount):
    results = ljm.eReadNames(handle, numFrames, names)
    plotResults.append(results)
    ljm.waitForNextInterval(intervalHandle)
    stamp = time.time()
    delay = stamp - start
    start = stamp

    print(delay, results)

In [ ]:
#To convert to pressure, multiply by 344738.0
conv_factor = 344738.0
pascal_to_psi = 1.45038E-4
print(Time(datetime.datetime.now()).isot)
print(f"Phigh = {results[0] * conv_factor:.1f} pascals, Plow = {results[1] * conv_factor:.1f} pascals")
print(f"Phigh = {results[0] * conv_factor * pascal_to_psi:.1f} psi,Plow = {results[1] * conv_factor * pascal_to_psi:.1f} psi")

In [ ]:
# Close handles
ljm.close(handle)

The commands below are useful if the device gets "locked up".

In [ ]:
ljm.closeAll()

In [ ]:
ljm.eStreamStop(handle)

In [ ]:
# Use this cell if we want pressure vs time.

pressures = []
loopAmount = 1000
conv_factor = 344738.0
pascal_to_psi = 1.45038E-4
# Read AIN's from the LabJack with eReadNames in a loop.
names = ["AIN0", "AIN1"]
numFrames = 2
intervalHandle = 1
ljm.startInterval(intervalHandle, 10000000)  # Delay between readings (in microseconds)
times = []
p1s = []
p2s = []
start = time.time()
for n in range(loopAmount):
    results = ljm.eReadNames(handle, numFrames, names)
    p1 = results[0] * conv_factor * pascal_to_psi
    p2 = results[1] * conv_factor * pascal_to_psi
    p1s.append(p1)
    p2s.append(p2)
    ljm.waitForNextInterval(intervalHandle)
    stamp = Time(datetime.datetime.now()).isot
    times.append(stamp)
    pressures.append([stamp, p1, p2])

    print(stamp, p1, p2)

In [ ]:
vMeas = 6.69
conv_factor = 344738.0
pascal_to_psi = 1.45038E-4
print(f"{vMeas} v is {vMeas*conv_factor} Pa or {vMeas*conv_factor*pascal_to_psi} PSI")